In [51]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
model_save_name = 'all_cnn_c.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

In [0]:

# model_save_name = 'all_cnn_c.pt'
# path = F"/content/gdrive/My Drive/{model_save_name}"
# model.load_state_dict(torch.load(path))

In [0]:
cuda = True
train_batch_size = 32
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, transforms

In [31]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class AllConvNet(nn.Module):
    def __init__(self, input_size, n_classes=10, **kwargs):
        super(AllConvNet, self).__init__()
        self.conv1 = nn.Conv2d(input_size, 96, 3, padding=1)
        self.conv2 = nn.Conv2d(96, 96, 3, padding=1)
        self.conv3 = nn.Conv2d(96, 96, 3, padding=1, stride=2)
        self.conv4 = nn.Conv2d(96, 192, 3, padding=1)
        self.conv5 = nn.Conv2d(192, 192, 3, padding=1)
        self.conv6 = nn.Conv2d(192, 192, 3, padding=1, stride=2)
        self.conv7 = nn.Conv2d(192, 192, 3)
        self.conv8 = nn.Conv2d(192, 192, 1)
        self.class_conv = nn.Conv2d(192, n_classes, 1)


    def forward(self, x):
        x_drop = F.dropout(x, .2)
        conv1_out = F.relu(self.conv1(x_drop))
        conv2_out = F.relu(self.conv2(conv1_out))
        conv3_out = F.relu(self.conv3(conv2_out))
        conv3_out_drop = F.dropout(conv3_out, .5)
        conv4_out = F.relu(self.conv4(conv3_out_drop))
        conv5_out = F.relu(self.conv5(conv4_out))
        conv6_out = F.relu(self.conv6(conv5_out))
        conv6_out_drop = F.dropout(conv6_out, .5)
        conv7_out = F.relu(self.conv7(conv6_out_drop))
        conv8_out = F.relu(self.conv8(conv7_out))

        class_out = F.relu(self.class_conv(conv8_out))
        pool_out = F.adaptive_avg_pool2d(class_out, 1)
        pool_out.squeeze_(-1)
        pool_out.squeeze_(-1)
        return pool_out

trial=AllConvNet(3)
print(trial)

AllConvNet(
  (conv1): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(96, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(192, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv7): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1))
  (conv8): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
  (class_conv): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
)


In [33]:
cuda =torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)

print(train_mean)
print(train_std)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
[0.49139968 0.48215841 0.44653091]
[0.24703223 0.24348513 0.26158784]


In [34]:
model = AllConvNet(3)
if cuda:
    model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9,weight_decay=0.001)
scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[200, 250, 300], gamma=0.1)
summary(model,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 32, 32]           2,688
            Conv2d-2           [-1, 96, 32, 32]          83,040
            Conv2d-3           [-1, 96, 16, 16]          83,040
            Conv2d-4          [-1, 192, 16, 16]         166,080
            Conv2d-5          [-1, 192, 16, 16]         331,968
            Conv2d-6            [-1, 192, 8, 8]         331,968
            Conv2d-7            [-1, 192, 6, 6]         331,968
            Conv2d-8            [-1, 192, 6, 6]          37,056
            Conv2d-9             [-1, 10, 6, 6]           1,930
Total params: 1,369,738
Trainable params: 1,369,738
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 2.64
Params size (MB): 5.23
Estimated Total Size (MB): 7.88
---------------------------------------

In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(epoch, best_loss, best_epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best.pt")
        # if gsync_save:
        #     gsync.update_file_to_folder("best.pt")
    return best_loss, best_epoch

In [0]:
for epoch in range(350):
    scheduler.step()
    train(epoch)
    best_loss, best_epoch = test(epoch, best_loss, best_epoch)

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.306128
Train Epoch: 0 [3200/50000 (6%)]	Loss: 2.302307
Train Epoch: 0 [6400/50000 (13%)]	Loss: 2.288123
Train Epoch: 0 [9600/50000 (19%)]	Loss: 2.307241
Train Epoch: 0 [12800/50000 (26%)]	Loss: 2.297220
Train Epoch: 0 [16000/50000 (32%)]	Loss: 2.305354
Train Epoch: 0 [19200/50000 (38%)]	Loss: 2.289232
Train Epoch: 0 [22400/50000 (45%)]	Loss: 2.328951
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.259106
Train Epoch: 0 [28800/50000 (58%)]	Loss: 2.281955
Train Epoch: 0 [32000/50000 (64%)]	Loss: 2.197603
Train Epoch: 0 [35200/50000 (70%)]	Loss: 2.263151
Train Epoch: 0 [38400/50000 (77%)]	Loss: 2.253476
Train Epoch: 0 [41600/50000 (83%)]	Loss: 2.203083
Train Epoch: 0 [44800/50000 (90%)]	Loss: 2.247094
Train Epoch: 0 [48000/50000 (96%)]	Loss: 2.157040

Test set: Average loss: 0.0178, Accuracy: 1820/10000 (18%)



/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type AllConvNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Streaming output truncated to the last 5000 lines.
Test set: Average loss: 0.0057, Accuracy: 7630/10000 (76%)

Train Epoch: 87 [0/50000 (0%)]	Loss: 0.549475
Train Epoch: 87 [3200/50000 (6%)]	Loss: 0.516751
Train Epoch: 87 [6400/50000 (13%)]	Loss: 0.515275
Train Epoch: 87 [9600/50000 (19%)]	Loss: 0.711593
Train Epoch: 87 [12800/50000 (26%)]	Loss: 0.734434
Train Epoch: 87 [16000/50000 (32%)]	Loss: 0.625560
Train Epoch: 87 [19200/50000 (38%)]	Loss: 0.715049
Train Epoch: 87 [22400/50000 (45%)]	Loss: 0.462746
Train Epoch: 87 [25600/50000 (51%)]	Loss: 0.589769
Train Epoch: 87 [28800/50000 (58%)]	Loss: 0.573494
Train Epoch: 87 [32000/50000 (64%)]	Loss: 0.523033
Train Epoch: 87 [35200/50000 (70%)]	Loss: 0.412174
Train Epoch: 87 [38400/50000 (77%)]	Loss: 0.716566
Train Epoch: 87 [41600/50000 (83%)]	Loss: 0.356317
Train Epoch: 87 [44800/50000 (90%)]	Loss: 0.478758
Train Epoch: 87 [48000/50000 (96%)]	Loss: 0.622121

Test set: Average loss: 0.0056, Accuracy: 7644/10000 (76%)

Train Epoch: 88 [0/50

In [50]:
data=[]
target=[]
for batch_idx, (data, target) in enumerate(test_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        # print(target)
        break
output=model(data)

test_loss = criterion(output, target).item()
# get the index of the max log-probability
pred = output.data.max(1, keepdim=True)[1]
#correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
print(pred.eq(target.data.view_as(pred)).long().sum())
print(len(target))

tensor(101, device='cuda:0')
124


In [37]:
# weight decay and 6x6
for epoch in range(350):
    train(epoch)
    scheduler.step()
    best_loss, best_epoch = test(epoch, best_loss, best_epoch)

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.301983
Train Epoch: 0 [3200/50000 (6%)]	Loss: 2.301054
Train Epoch: 0 [6400/50000 (13%)]	Loss: 2.301573
Train Epoch: 0 [9600/50000 (19%)]	Loss: 2.302974
Train Epoch: 0 [12800/50000 (26%)]	Loss: 2.302094
Train Epoch: 0 [16000/50000 (32%)]	Loss: 2.302994
Train Epoch: 0 [19200/50000 (38%)]	Loss: 2.303131
Train Epoch: 0 [22400/50000 (45%)]	Loss: 2.302647
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.302122
Train Epoch: 0 [28800/50000 (58%)]	Loss: 2.302185
Train Epoch: 0 [32000/50000 (64%)]	Loss: 2.303130
Train Epoch: 0 [35200/50000 (70%)]	Loss: 2.303131
Train Epoch: 0 [38400/50000 (77%)]	Loss: 2.302808
Train Epoch: 0 [41600/50000 (83%)]	Loss: 2.301012
Train Epoch: 0 [44800/50000 (90%)]	Loss: 2.300075
Train Epoch: 0 [48000/50000 (96%)]	Loss: 2.302172

Test set: Average loss: 0.0187, Accuracy: 1000/10000 (10%)



/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type AllConvNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Streaming output truncated to the last 5000 lines.
Test set: Average loss: 0.0072, Accuracy: 6943/10000 (69%)

Train Epoch: 87 [0/50000 (0%)]	Loss: 0.677079
Train Epoch: 87 [3200/50000 (6%)]	Loss: 1.069675
Train Epoch: 87 [6400/50000 (13%)]	Loss: 1.201721
Train Epoch: 87 [9600/50000 (19%)]	Loss: 0.625064
Train Epoch: 87 [12800/50000 (26%)]	Loss: 1.063762
Train Epoch: 87 [16000/50000 (32%)]	Loss: 0.991517
Train Epoch: 87 [19200/50000 (38%)]	Loss: 0.719829
Train Epoch: 87 [22400/50000 (45%)]	Loss: 0.587492
Train Epoch: 87 [25600/50000 (51%)]	Loss: 1.099468
Train Epoch: 87 [28800/50000 (58%)]	Loss: 0.721424
Train Epoch: 87 [32000/50000 (64%)]	Loss: 0.483652
Train Epoch: 87 [35200/50000 (70%)]	Loss: 0.817203
Train Epoch: 87 [38400/50000 (77%)]	Loss: 0.876345
Train Epoch: 87 [41600/50000 (83%)]	Loss: 0.898934
Train Epoch: 87 [44800/50000 (90%)]	Loss: 0.702936
Train Epoch: 87 [48000/50000 (96%)]	Loss: 1.074179

Test set: Average loss: 0.0073, Accuracy: 6849/10000 (68%)

Train Epoch: 88 [0/50